In [1]:
########
# Just input the complete sequence tokenized, and output logits and reconstruct the sequence
# Are there errors there?
########

In [3]:
from tqdm import tqdm
import random
import os
import argparse
import numpy as np
import pandas as pd
from Bio import SeqIO
import re
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import json

# Load the tokenizer
model = "models/prottrans_t5_xl_u50/"
tokenizer = T5Tokenizer.from_pretrained(model, do_lower_case=False, legacy=False)
torch.cuda.empty_cache()

In [10]:
fastafile = "disprot_OK_fullset.fasta"

counter = 0
match_sequence = list()
uniprots_read = list()
for record in tqdm(SeqIO.parse(fastafile, "fasta")):
    uniprot_id = record.id
    uniprots_read.append(uniprot_id)
    aa_sequence = str(record.seq)
    
    target_seq = aa_sequence
    input_seq = [" ".join(list(re.sub(r"[UZOB]", "X", target_seq)))]

    ## Load logits from file
    cpulogits = torch.load(f"logits/{uniprot_id}_logits.pt") 
    fastpred = tokenizer.decode(torch.tensor(cpulogits[:,:-1,:].numpy().argmax(-1)[0]), skip_special_tokens=False).replace("<"," <").replace(">","> ")
    if input_seq[0] == fastpred:
        match_sequence.append(True)
    else:
        pred_arr = fastpred.split()
        seq_arr  = input_seq[0].split()
        if len(pred_arr) == len(seq_arr):
            local_match_sequence = list()
            for j in range(len(pred_arr)):
                if pred_arr[j] != seq_arr[j]:
                    local_match_sequence.append((j,pred_arr[j], seq_arr[j]))
        else:
            print("Mismatch length error")
            raise
        match_sequence.append(local_match_sequence)

2145it [00:37, 57.72it/s]


In [12]:
print(match_sequence)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, Tru